# Preprocessing: BJH051


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH051'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH051/ieeg/BJH051_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH051/ieeg/BJH051_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 3179999 =   1043.000 ...  1589.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH051/ieeg/BJH051_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH051/ieeg/BJH051_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 3179999 =   1043.000 ...  1589.999 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AL1  noref
AL2  noref
AL3  noref
AL4 AL5
REF1 noref
REF2 noref
AL5 AL6
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL10
AL10 AL11
AL11 AL12
BL1 noref
BL2 noref
BL3 noref
BL4 noref
BL5 noref
BL6 BL7
BL7 BL8
BL8 BL9
BL9 BL10
BL10 BL11
BL11 BL12
CL1 noref
CL2 noref
CL3 noref
CL4 noref
CL5 noref
CL6 CL7
CL7 CL8
CL8 CL9
CL9 CL10
CL10 CL11
CL11 CL12
DL1 noref
DL2 noref
DL3 noref
DL4 noref
DL5 noref
DL6 noref
DL7 noref
DL8 noref
DL9 DL10
DL10 DL11
DL11 DL12
EL1 noref
EL2 noref
EL3 noref
EL4 noref
EL5 noref
EL6 EL7
EL7 EL8
FL1 noref
FL2 noref
FL3 noref
FL4 noref
FL5 noref
FL6 noref
FL7 noref
FL8 noref
FL9 FL10
FL11 noref
FL12 noref
GL1 GL2
GL2 GL3
GL3 GL4
GL4 GL5
GL5 GL6
GL6 GL7
GL7 GL8
GL8 GL9
GL9 GL10
GL10 GL11
GL11 GL12
HL1 HL2
HL2 HL3
HL3 HL4
HL4 HL5
HL5 HL6
HL6 HL7
HL7 HL8
HL8 HL9
HL9 HL10
IL1 IL2
IL2 IL3
IL3 IL4
IL4 IL5
IL5 IL6
IL6 IL7
IL7 IL8
JL1 JL2
JL2 JL3
JL3 JL4
JL4 JL5
JL5 JL6
JL6 JL7
JL7 JL8
JL8 JL9
JL9 JL10
KL1 KL2
KL2 KL3
KL3 KL4
KL4 KL5
KL5 KL6
KL6 KL7
KL7 KL8
KL8 KL9
KL9 KL10
KL10 KL11
KL11 K

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 3179999  =      0.000 ...  1589.999 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=129, n_times=3180000
    Range : 0 ... 3179999 =      0.000 ...  1589.999 secs
Ready.
Added the following bipolar channels:
AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, DL9-DL10, DL10-DL11, DL11-DL12, EL6-EL7, EL7-EL8, FL9-FL10, GL1-GL2, GL2-GL3, GL3-GL4, GL4-GL5, GL5-GL6, GL6-GL7, GL7-GL8, GL8-GL9, GL9-GL10, GL10-GL11, GL11-GL12, HL1-HL2, HL2-HL3, HL3-HL4, HL4-HL5, HL5-HL6, HL6-HL7, HL7-HL8, HL8-HL9, HL9-HL10, IL1-IL2, IL2-IL3, IL3-IL4, IL4-IL5, IL5-IL6, IL6-IL7, IL7-IL8, JL1-JL2, JL2-JL3, JL3-JL4, JL4-JL5, JL5-JL6, JL6-JL7, JL7-JL8, JL8-JL9, JL9-JL10, KL1-KL2, KL2-KL3, KL3-KL4, KL4-KL5, KL5-KL6, KL6-KL7, KL7-KL8, KL8-KL9, KL9-KL10, KL10-KL11, KL11-KL12, LL1-LL2, LL2-LL3, LL3-LL4, LL4-LL5

/tmp/ipykernel_6133/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif
Writing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif
[done]
Reading 0 ... 3179999  =      0.000 ...  1589.999 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=129, n_times=3180000
    Range : 0 ... 3179999 =      0.000 ...  1589.999 secs
Ready.
Added the following bipolar channels:
AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, DL9-DL10, DL10-DL11, DL11-DL12, EL6-EL7, EL7-EL8, FL9-FL10, GL1-GL2, GL2-GL3, GL3-GL4, GL4-GL5, GL5-GL6, GL6-GL7, GL7-GL8, GL8-GL9, GL9-GL10, GL10-GL11, GL11-GL12, HL1-HL2, HL2-HL3, HL3-HL4, HL4-HL5, HL5-HL6, HL6-HL7, HL7-HL8, HL8-HL9, HL9-HL10, IL1-IL2,

/tmp/ipykernel_6133/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_raw_clean_data.fif
Writing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_raw_clean_data-1.fif
[done]


In [ ]:
# Visualize it #

# bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [7]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.


/tmp/ipykernel_6133/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 32001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 32001 original time points ...


/tmp/ipykernel_6133/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 32001 original time points ...


### Trial End

In [9]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1] # remove last event
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.


/tmp/ipykernel_6133/3432568909.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Overwriting existing file.


/tmp/ipykernel_6133/3432568909.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...
Overwriting existing file.
Loading data for 119 events and 20001 original time points ...


### Last Away

In [10]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,44000,0,1
1,2,20,82500,0,1
2,3,4,98500,0,1
3,4,12,112300,0,1
4,6,2,137600,0,1
...,...,...,...,...,...
211,232,18,2903300,0,1
212,233,1,2914300,0,1
213,234,15,2924000,0,1
214,236,10,2946900,0,1


In [11]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [12]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.
Not setting metadata
216 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 108 events and 20001 original time points ...


/tmp/ipykernel_6133/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6133/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 108 events and 20001 original time points ...


### First Dot

In [13]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,43300,0,1
1,1,1,53900,0,1
2,2,20,67700,0,1
3,3,4,96400,0,1
4,4,12,110100,0,1
...,...,...,...,...,...
223,234,15,2922300,0,1
224,235,11,2932600,0,1
225,236,10,2945300,0,1
226,237,4,2955200,0,1


In [14]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [15]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.
Not setting metadata
228 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 114 events and 18001 original time points ...


/tmp/ipykernel_6133/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6133/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 114 events and 18001 original time points ...


## First Move

In [16]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,39700,0,1
1,1,1,53300,0,1
2,2,20,67300,0,1
3,3,4,96100,0,1
4,4,12,109200,0,1
...,...,...,...,...,...
234,234,15,2922100,0,1
235,235,11,2932000,0,1
236,236,10,2944700,0,1
237,237,4,2954800,0,1


In [17]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [18]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_6133/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6133/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 119 events and 20001 original time points ...


## Ghost Attack

In [19]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,11,43400,0,1
1,1,1,55800,0,1
2,5,3,125200,0,1
3,8,10,161800,0,1
4,10,6,210700,0,1
5,15,13,296000,0,1
6,17,15,315400,0,1
7,19,7,338400,0,1
8,27,15,459400,0,1
9,28,6,479100,0,1


In [20]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [21]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif...
    Range : 0 ... 2225999 =      0.000 ...  1112.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data-1.fif...
    Range : 2226000 ... 3179999 =   1113.000 ...  1589.999 secs
Ready.
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 45 events and 20001 original time points ...


/tmp/ipykernel_6133/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_6133/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH051/ieeg/BJH051_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
